# Monte Carlo Relaxation

In [ ]:
from __future__ import print_function

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import random, math

## Setup perfect system

In [ ]:
from lammps import IPyLammps

In [ ]:
L = IPyLammps()

In [ ]:
L.units("lj")
L.atom_style("atomic")
L.atom_modify("map array sort", 0, 0.0)

L.dimension(2)

L.lattice("hex", 1.0)
L.region("box block", 0, 10, 0, 5, -0.5, 0.5)

L.create_box(1, "box")
L.create_atoms(1, "box")
L.mass(1, 1.0)

L.pair_style("lj/cut", 2.5)
L.pair_coeff(1, 1, 1.0, 1.0, 2.5)
L.pair_modify("shift", "yes")

L.neighbor(0.3, "bin")
L.neigh_modify("delay", 0, "every", 1, "check", "yes")

In [ ]:
L.image(zoom=1.6)

In [ ]:
L.run(0);

In [ ]:
emin = L.eval("pe")

In [ ]:
L.dump("3 all movie 25 movie.mp4 type type zoom 1.6 adiam 1.0")

## Disorder system

In [ ]:
random.seed(27848)
deltaperturb = 0.2

In [ ]:
for i in range(L.system.natoms):
    x, y = L.atoms[i].position
    dx = deltaperturb * random.uniform(-1, 1)
    dy = deltaperturb * random.uniform(-1, 1)
    L.atoms[i].position = (x+dx, y+dy)

In [ ]:
L.run(0);

In [ ]:
L.image(zoom=1.6)

## Minimize using Monte Carlo moves

In [ ]:
estart = L.eval("pe")
elast = estart

In [ ]:
naccept = 0

In [ ]:
energies = [estart]

In [ ]:
niterations = 3000
deltamove = 0.1
kT = 0.05

In [ ]:
natoms = L.system.natoms

for i in range(niterations):
    iatom = random.randrange(0, natoms)
    current_atom = L.atoms[iatom]
    
    x0, y0 = current_atom.position
    
    dx = deltamove * random.uniform(-1, 1)
    dy = deltamove * random.uniform(-1, 1)
    
    current_atom.position = (x0+dx, y0+dy)
    
    L.run(1, "pre no post no")
    
    e = L.eval("pe")
    energies.append(e)
    
    if e <= elast:
        naccept += 1
        elast = e
    elif random.random() <= math.exp(natoms*(elast-e)/kT):
        naccept += 1
        elast = e
    else:
        current_atom.position = (x0, y0)

In [ ]:
plt.xlabel('iteration')
plt.ylabel('potential energy')
plt.plot(energies)

In [ ]:
L.eval("pe")

In [ ]:
emin

In [ ]:
estart

In [ ]:
naccept

In [ ]:
L.image(zoom=1.6)

In [ ]:
# close dump file to access it
L.undump(3)

In [ ]:
L.video("movie.mp4")